In [1]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling1D, Dense, Activation, Dropout, Embedding, Conv1D
import random

def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename =r"C:\Users\Rohan\Pictures\rohan\langchine\hh.json"

intents = load_json_file(filename) 

df = pd.DataFrame(columns=['Pattern', 'Tag'])

def extract_json_info(json_file, df):
    for intent in json_file['intents']:
        for pattern in intent['patterns']:
            sentence_tag = [pattern, intent['tag']]
            df.loc[len(df.index)] = sentence_tag
    return df

df = extract_json_info(intents, df) 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Pattern'])

X = tokenizer.texts_to_sequences(df['Pattern'])

max_sequence_length = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_sequence_length, padding='post')

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Tag'])

from keras.models import load_model
loaded_model = load_model('chatbot.h5')
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    input_sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents):
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    predicted_tag = label_encoder.inverse_transform(predicted_label)
    for intent in intents["intents"]:
        if intent["tag"] == predicted_tag[0]:
            response = random.choice(intent["responses"])
            return response
    
    return "I'm sorry, I didn't understand that."


print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    sentence = input("You: ")
    if sentence.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents)
    print("Chatbot:", response)



Chatbot: Hello! How can I assist you today? (Type 'exit' to end)
You: exit
Chatbot: Goodbye! Have a great day!
